<a href="https://colab.research.google.com/github/IshuDhana/lab-babyagi-with-agent/blob/main/BabyAGI_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Lab | BabyAGI with agent

Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats You can't get this wrong!

You would need the OpenAI API KEY and the SerpAPI KEY to run this lab.
BabyAGI with Tools

This notebook builds on top of baby agi, but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information


In [1]:
# ============================================================
# INSTALL REQUIRED PACKAGES
# ============================================================
# We install a specific set of LangChain packages that work together.
# google-search-results is required for SerpAPI search.
!pip install -q langchain langchain-openai langchain-core langchain-community \
                  langchain-experimental langchain-classic python-dotenv \
                  faiss-cpu google-search-results


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# ============================================================
# IMPORTS
# ============================================================

import os
import sys
import getpass
from dotenv import load_dotenv, find_dotenv

# OpenAI + Embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Tools + Document schema
from langchain_core.tools import Tool
from langchain_core.documents import Document

# BabyAGI (experimental LangChain agent)
from langchain_experimental.autonomous_agents import BabyAGI

# Vector store
from langchain_community.vectorstores import FAISS

# Search tool
from langchain_community.utilities import SerpAPIWrapper

# Classic agent wrapper
from langchain_classic.agents import initialize_agent, AgentType


In [3]:
# ============================================================
# LOAD ENVIRONMENT VARIABLES
# ============================================================
# If .env file exists, load it. Otherwise user inputs API keys manually.

load_dotenv(find_dotenv())

# Enter API keys interactively (safer for public notebooks)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")
os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your SERPAPI_API_KEY: ")


Enter your OPENAI_API_KEY: ··········
Enter your SERPAPI_API_KEY: ··········


In [4]:
# ============================================================
# INITIALIZE LLM + EMBEDDINGS
# ============================================================
# ChatOpenAI uses the OpenAI Chat Completions API.
# model="gpt-3.5-turbo" is low-cost and works well for BabyAGI examples.

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Embeddings used by FAISS for vector search
embeddings_model = OpenAIEmbeddings()


In [5]:
# ============================================================
# CREATE VECTORSTORE WITH EXAMPLE DOCUMENTS
# ============================================================
# BabyAGI uses a vector database to store and recall tasks.
# Here we store generic "task descriptions" as example documents.

documents = [
    Document(page_content="Create comprehensive plans", metadata={"task": "Planning"}),
    Document(page_content="Research information", metadata={"task": "Research"}),
    Document(page_content="Generate schedules", metadata={"task": "Scheduling"}),
    Document(page_content="Create checklists",  metadata={"task": "Organization"}),
]

# Build the FAISS vector index
vectorstore = FAISS.from_documents(documents, embeddings_model)


In [6]:
# ============================================================
# DEFINE TOOLS (FUNCTIONS BABYAGI CAN USE)
# ============================================================
# Each tool wraps an LLM or external call (like SerpAPI).
# Tools allow BabyAGI to perform actions.

# ---- 1. PLANNING TOOL ----
def create_plan(objective: str) -> str:
    """Generate a detailed plan for a given objective."""
    response = llm.invoke(
        f"Create a detailed step-by-step plan for: {objective}.\n"
        f"Include milestones, timeline, and required resources."
    )
    return response.content

# ---- 2. SEARCH TOOL ----
def research_topic(query: str) -> str:
    """Run a SerpAPI search or return a fallback message."""
    try:
        search = SerpAPIWrapper()
        return search.run(query)
    except:
        return f"Basic research result for: {query}"

# ---- 3. SCHEDULE TOOL ----
def generate_schedule(objective: str) -> str:
    """Generate a multi-day schedule for an objective."""
    response = llm.invoke(
        f"Create a day-by-day schedule for: {objective}"
    )
    return response.content

# ---- 4. CHECKLIST TOOL ----
def create_checklist(objective: str) -> str:
    """Create a clean and organized checklist."""
    response = llm.invoke(
        f"Create a detailed checklist for: {objective}"
    )
    return response.content

# ---- PACKAGE TOOLS INTO LANGCHAIN TOOL OBJECTS ----
tools = [
    Tool(name="Research",   func=research_topic,   description="Search for information online."),
    Tool(name="Planner",    func=create_plan,      description="Create detailed plans."),
    Tool(name="Scheduler",  func=generate_schedule, description="Generate multi-day schedules."),
    Tool(name="Checklist",  func=create_checklist, description="Create organized checklists."),
]


In [7]:
# ============================================================
# CREATE TRADITIONAL AGENT (OPTIONAL)
# ============================================================
# This is not BabyAGI yet – this is a reactive agent that can use tools.
# BabyAGI will call tools internally, but we show both for clarity.

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5
)


/tmp/ipython-input-4101296421.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(


In [8]:
# ============================================================
# CREATE BABYAGI INSTANCE
# ============================================================
# BabyAGI uses:
#  - The LLM
#  - A vectorstore to store tasks/results
#  - Automatically generated tasks
#  - max_iterations to prevent infinite loops

baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    verbose=False,
    max_iterations=3
)


In [9]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")



🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> yes

************************************************************
OBJECTIVE: yes
************************************************************



In [10]:
# ============================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****

1. Research new project ideas
2. Plan out project timelines and goals
3. Organize project resources and team members
4. Schedule meetings and check-ins with team members
5. Create a detailed task list for each project phase
6. Set deadlines and milestones for project completion
7. Monitor progress and adjust plans as needed
8. Communicate updates and next steps to team members.

*****TASK LIST*****

1: Develop a project proposal outlining the scope, objectives, and expected outcomes
2: Brainstorm potential project ideas based on current market trends and industry needs
3: Conduct market research to identify potential gaps and opportunities for new projects
4: Identify key stakeholders and potential team members for the project
5: Create a project budget and resource allocation plan
6: Develop a project communication plan to keep stakeholders informed and engaged
7: Conduct a risk

following are the exmaples you can try:
# Example 1: Travel Planning
"Plan a 5-day trip to Italy including accommodations, activities, and budget"

# Example 2: Project Management
"Create a launch plan for my new product including timeline, marketing, and resources"

# Example 3: Personal Planning
"Plan a healthy meal prep for the next 2 weeks with shopping list and recipes"

# Example 4: Event Planning
"Organize a wedding for 100 guests in 3 months including venue, catering, and timeline"

# Example 5: Learning
"Create a 30-day plan to learn Python programming from scratch"